In [8]:
from flask import Flask, render_template
import folium
import geopandas as gpd
import requests

app = Flask(__name__)

In [9]:
# Download Virginia state boundary
virginia_state_url = "https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/cultural/ne_10m_admin_1_states_provinces.zip"
virginia_state_file = "ne_10m_admin_1_states_provinces.shp"

# Download and extract the shapefile
r = requests.get(virginia_state_url)
with open(virginia_state_file.replace(".shp", ".zip"), "wb") as f:
    f.write(r.content)

# Load Virginia state boundary
virginia_state = gpd.read_file(virginia_state_file)[
    gpd.read_file(virginia_state_file)["name"] == "Virginia"
]

# Download Virginia counties
virginia_counties_url = "https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/cultural/ne_10m_admin_1_states_provinces_lakes.zip"
virginia_counties_file = "ne_10m_admin_1_states_provinces_lakes.shp"

# Download and extract the shapefile
r = requests.get(virginia_counties_url)
with open(virginia_counties_file.replace(".shp", ".zip"), "wb") as f:
    f.write(r.content)

# Load Virginia counties
virginia_counties = gpd.read_file(virginia_counties_file)[
    gpd.read_file(virginia_counties_file)["admin"] == "Virginia"
]

DriverError: ne_10m_admin_1_states_provinces.shp: No such file or directory

In [3]:
# Download Virginia state boundary
virginia_state = gpd.datasets.get_path("naturalearth_lowres")

# Download Virginia counties
virginia_counties = gpd.datasets.get_path("naturalearth_lowres", "us_counties")

/var/folders/ch/2jdk3nmx6m59lq348r2ydl3m0000gp/T/ipykernel_3988/808591537.py:2: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  virginia_state = gpd.datasets.get_path("naturalearth_lowres")


TypeError: get_path() takes 1 positional argument but 2 were given

In [ ]:
# Load cities to highlight
cities = ["CHARLOTTESVILLE", "ROANOKE", "LYNCHBURG", "LEXINGTON"]
city_coords = {
    "CHARLOTTESVILLE": (38.0, -78.5),
    "ROANOKE": (37.2, -79.9),
    "LYNCHBURG": (37.4, -79.1),
    "LEXINGTON": (37.7, -78.5),
}

In [ ]:
# Create the map
map = folium.Map(location=[37.5, -78.3], zoom_start=7)

In [ ]:
# Add Virginia state boundaries and counties
virginia_states["geometry"].apply(
    lambda x: folium.Polygon(x.exterior.coords[:], color="blue", weight=2)
).tolist().append(map)

In [ ]:
# Highlight cities
for city, coords in city_coords.items():
    folium.CircleMarker(coords, radius=5, color='black').add_to(map)
    folium.Marker([coords[0], coords[1]], popup=city).add_to(map)

In [ ]:
# Create the Flask app and render the map
@app.route('/')
def index():
    return render_template('index.html', map=map._repr_html_())

if __name__ == '__main__':
    app.run(debug=True)